### Plan

I wil try to reach a base line model first using an LSTM basic implemenation with the same data processing done on the Feed Forward Neural Networks

In [2]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..')) # Adjust '..' if your notebook is deeper
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
from models.src.preprocessing.tokenizer import tokenize_data_frame
from data.process_data_modeling import get_processed_data
from sklearn.model_selection import train_test_split
import numpy as np
import logging
import nltk
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import trange
from collections import Counter
import matplotlib.pyplot as plt
embedding_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
from torch.utils.data import IterableDataset, DataLoader

/Users/cerdricdamais/Desktop/EPITA/MAJEUR/NLP-1/NLP_Linkedin_offers/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cerdricdamais/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cerdricdamais/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2025-05-07 20:59:44,040 - INFO - Use pytorch device_name: mps
2025-05-07 20:59:44,041 - INFO - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-05-07 20:59:45,181 - WARNING - Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK 'punkt' tokenizer...")
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading NLTK 'stopwords'...")
    nltk.download('stopwords')

# Add this check and download for punkt_tab
try:
    # Check for the specific English directory within punkt_tab
    nltk.data.find('tokenizers/punkt_tab/english/') 
except LookupError:
    print("Downloading NLTK 'punkt_tab'...")
    nltk.download('punkt_tab')

print("NLTK resources checked/downloaded.")



nltk.download('punkt')
nltk.download('stopwords')

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


NLTK resources checked/downloaded.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cerdricdamais/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cerdricdamais/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

Using device: cpu
GPU available: False


In [8]:
df = get_processed_data()
df.head()
df.drop(columns=["company_name", "location"], inplace=True)
df.head()

2025-05-07 21:02:07,597 - INFO - Importing data from Kaggle
2025-05-07 21:02:07,599 - INFO - Path to dataset files: /Users/cerdricdamais/.cache/kagglehub/datasets/arshkon/linkedin-job-postings/versions/13
2025-05-07 21:02:07,600 - INFO - List of files in the dataset: ['postings.csv', 'mappings', 'jobs', 'companies']
2025-05-07 21:02:11,739 - INFO - Rows with at least one NaN value: 1725
2025-05-07 21:02:11,741 - INFO - Number of rows before dropping NaN values: 123849
2025-05-07 21:02:11,772 - INFO - Number of rows after dropping NaN values: 122124
2025-05-07 21:03:01,598 - INFO - DataFrame saved to: /Users/cerdricdamais/Desktop/EPITA/MAJEUR/NLP-1/NLP_Linkedin_offers/models/src/generation/data/processed/cleaned_postings_modeling.parquet
2025-05-07 21:03:01,603 - INFO - Data processing completed successfully !
2025-05-07 21:03:01,603 - INFO - Returning the processed DataFrame


,title,description
0,Marketing Coordinator,job description a leading real estate firm in ...
2,Assitant Restaurant Manager,the national exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,senior associate attorney elder law trusts and...
5,Economic Development and Planning Intern,job summary the economic development planning ...
6,Producer,company description raw cereal is a creative d...


In [14]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.head()

,title,description
88431,Big Spring Market Overnight Perishables Rep Pa...,h e b needs energetic and motivated partners w...
94806,Front Desk Guest Services Representative,driftwood hospitality management s company cul...
67484,AVP Head of National Markets,"job description providing for loved ones, plan..."
29134,Air Export Operations Specialist,well know logistics company based on the nassa...
24667,Outside Sales Representative Medical Equipment,as america s largest supplier of respiratory a...


In [9]:
def remove_stop_words(df, column_name='merged'):
    """
    Removes predefined unwanted tokens from a specified text column in the DataFrame.
    It splits the text by whitespace, filters out tokens present in the unwanted_set,
    and then rejoins the tokens.
    The modification is done in-place.
    """
    unwanted_token = {"-", ".", "!", "?", "...", "....", ".....", "/", "|", "~", "`", "=", "+", "_", "*"}

    def clean_text_series(text_series):
        if not isinstance(text_series, str):
            return text_series  # Return as is if not a string (e.g., NaN, numbers)
        
        tokens = text_series.split()
        
        # Filter out tokens that are exactly in unwanted_token
        filtered_tokens = [token for token in tokens if token not in unwanted_token]
        
        return " ".join(filtered_tokens)
    df[column_name] = df[column_name].apply(clean_text_series)
        

remove_stop_words(df, column_name="title")
remove_stop_words(df, column_name="description")
df.head()

,title,description
0,Marketing Coordinator,job description a leading real estate firm in ...
2,Assitant Restaurant Manager,the national exemplar is accepting application...
3,Senior Elder Law Trusts and Estates Associate ...,senior associate attorney elder law trusts and...
5,Economic Development and Planning Intern,job summary the economic development planning ...
6,Producer,company description raw cereal is a creative d...


In [10]:
import pickle

# Configure logging if not already configured
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# --- This is the function from the previous response, kept for completeness ---
def create_all_title_embeddings_cache(list_of_all_titles, sentence_transformer_model, cache_file_path):
    logging.info(f"Generating embeddings for {len(list_of_all_titles)} unique titles...")
    title_embeddings_array = sentence_transformer_model.encode(
        list_of_all_titles, batch_size=32, show_progress_bar=True, convert_to_tensor=False
    )
    with open(cache_file_path, 'wb') as f:
        pickle.dump(title_embeddings_array, f)
    logging.info(f"Title embeddings for {len(list_of_all_titles)} titles saved to {cache_file_path}")
    return title_embeddings_array


class OptionBDataLoader:
    def __init__(self, dataframe,
                 sentence_transformer_model,
                 unique_titles_embedding_cache_path,
                 description_tokenizer_func,
                 description_word_to_idx,
                 batch_size,
                 device='cuda'):

        self.dataframe = dataframe
        self.sentence_transformer_model = sentence_transformer_model
        self.unique_titles_embedding_cache_path = unique_titles_embedding_cache_path
        self.description_tokenizer_func = description_tokenizer_func
        
        # Ensure your vocabulary map is passed correctly
        self.word_to_idx = description_word_to_idx
        self.idx_to_word = {idx: word for word, idx in description_word_to_idx.items()}
        
        self.batch_size = batch_size
        self.device = device

        # Fetch IDs for special tokens from YOUR vocabulary map
        # It's crucial that your `description_word_to_idx` contains these keys.
        self.pad_idx = self.word_to_idx['<PAD>']
        self.start_idx = self.word_to_idx['__START__'] # Using your specified token
        self.end_idx = self.word_to_idx['__END__']     # Using your specified token
        self.unk_idx = self.word_to_idx['<UNK>']

        # --- Title Embeddings Handling (same as before) ---
        self.unique_titles = sorted(self.dataframe['title'].unique().tolist())
        self.title_to_idx_map = {title: i for i, title in enumerate(self.unique_titles)}
        try:
            with open(self.unique_titles_embedding_cache_path, 'rb') as f:
                self.unique_title_embeddings_array = pickle.load(f)
            logging.info(f"Loaded pre-computed embeddings for {len(self.unique_title_embeddings_array)} unique titles from {self.unique_titles_embedding_cache_path}")
            if len(self.unique_title_embeddings_array) != len(self.unique_titles):
                logging.warning("Mismatch in length of cached embeddings and unique titles. Recomputing.")
                raise FileNotFoundError
        except FileNotFoundError:
            logging.info(f"Cache not found or mismatch. Creating embeddings for {len(self.unique_titles)} unique titles...")
            self.unique_title_embeddings_array = create_all_title_embeddings_cache(
                self.unique_titles, self.sentence_transformer_model, self.unique_titles_embedding_cache_path
            )
        self.unique_title_embeddings_tensor = torch.tensor(self.unique_title_embeddings_array, dtype=torch.float)
        self.title_indices_for_df = [self.title_to_idx_map[title] for title in self.dataframe['title']]
        
        # --- Prepare all description data for iteration ---
        self.tokenized_descriptions = []
        for desc_text in self.dataframe['description']:
            # Tokenize the raw description text
            tokens_from_desc = self.description_tokenizer_func(desc_text)
            
            # Convert tokens to IDs, using <UNK> for out-of-vocab tokens
            numericalized_tokens = [self.word_to_idx.get(token, self.unk_idx) for token in tokens_from_desc]
            
            # Add __START__ at the beginning and __END__ at the end
            final_sequence_ids = [self.start_idx] + numericalized_tokens + [self.end_idx]
            self.tokenized_descriptions.append(torch.tensor(final_sequence_ids, dtype=torch.long))

        self.dataset_indices = list(range(len(self.dataframe)))

    def _pad_sequences(self, sequences_batch, max_len=None):
        # (Same as before - pads sequences in a batch to the same length)
        if max_len is None:
            max_len = max(len(seq) for seq in sequences_batch)
        
        padded_batch = torch.full((len(sequences_batch), max_len), self.pad_idx, dtype=torch.long)
        for i, seq in enumerate(sequences_batch):
            length = min(len(seq), max_len)
            padded_batch[i, :length] = seq[:length]
        return padded_batch

    def generate_batches(self):
        # (Same as before - generates batches of title_embeddings, decoder_inputs, decoder_targets)
        current_order_indices = list(self.dataset_indices)
        np.random.shuffle(current_order_indices)

        for i in range(0, len(current_order_indices), self.batch_size):
            batch_df_indices = current_order_indices[i : i + self.batch_size]
            
            batch_title_unique_indices = [self.title_indices_for_df[df_idx] for df_idx in batch_df_indices]
            batch_title_embeddings = self.unique_title_embeddings_tensor[batch_title_unique_indices].to(self.device)
            
            batch_desc_sequences = [self.tokenized_descriptions[df_idx] for df_idx in batch_df_indices]
            
            # Decoder input: e.g., [__START__, w1, w2]
            # Decoder target: e.g., [w1, w2, __END__]
            # These are created from the full sequence: [__START__, w1, w2, __END__]
            decoder_input_batch = self._pad_sequences([seq[:-1] for seq in batch_desc_sequences])
            decoder_target_batch = self._pad_sequences([seq[1:] for seq in batch_desc_sequences])
            
            yield batch_title_embeddings, decoder_input_batch.to(self.device), decoder_target_batch.to(self.device)

In [7]:
class EncoderLSTM(nn.Module):
    def __init__(self):
        pass
    
    def forward(self):
        pass

class DecoderLSTM(nn.Module):
    def __init__(self, encoder_out_dim):
        self.encoder_out_dim = encoder_out_dim
        self.decoder_in_dim = encoder_out_dim
    
    def forward(self):
        pass

In [ ]:
class LSTMLanguageModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, lr, layers) -> None:
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.rnn = self.init_model()
        self.decoder = DecoderLSTM()
        self.encoder = EncoderLSTM()
    
    def init_model(self):
        pass    

    def train():
        pass
    

        


In [ ]:
class DecoderLSTMWithAttention(nn.Module):
    def __init__(self):
        pass

In [11]:
class LSTMLanguageModelT(nn.Module):
    def __init__(self, 
                 embedding_model,                 # SentenceTransformer
                 decoder_vocab_size,              # Size of your description vocabulary
                 embedding_dim=384,               # Dimension of Transformer embeddings
                 decoder_embed_dim=256,           # Dimension for your decoder token embeddings
                 hidden_dim=512,                  # LSTM hidden dimension
                 num_layers=2,                    # Number of LSTM layers
                 dropout_p=0.5):                  # Dropout probability
        super(LSTMLanguageModelT, self).__init__()  # Don't forget this!
        
        # Store the embedding model
        self.title_embedding_transformer = embedding_model
        # Freeze the transformer parameters
        for param in self.title_embedding_transformer.parameters():
            param.requires_grad = False
            
        # Dimensions
        self.embedding_dim = embedding_dim        # Dimension from the transformer
        self.decoder_embed_dim = decoder_embed_dim  # Dimension for description tokens
        self.hidden_dim = hidden_dim              # LSTM hidden state size
        self.num_layers = num_layers
        self.decoder_vocab_size = decoder_vocab_size
        
        # Linear projections for initializing LSTM states from title embedding
        self.title_to_hidden = nn.Linear(embedding_dim, hidden_dim)
        self.title_to_cell = nn.Linear(embedding_dim, hidden_dim)
        
        # Embedding layer for decoder input tokens (descriptions)
        self.decoder_embedding = nn.Embedding(decoder_vocab_size, decoder_embed_dim)
        
        # LSTM for generating the description token by token
        self.lstm = nn.LSTM(input_size=decoder_embed_dim,
                           hidden_size=hidden_dim,
                           num_layers=num_layers,
                           batch_first=True,
                           dropout=dropout_p if num_layers > 1 else 0)
        
        # Final output projection
        self.output_projection = nn.Linear(hidden_dim, decoder_vocab_size)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(dropout_p)
    
    def _get_title_embeddings(self, title_texts):
        """Get embeddings for a batch of titles using the transformer"""
        with torch.no_grad():  # No need to track gradients for the frozen transformer
            # The .encode() method returns numpy arrays, convert to tensors
            title_embeddings_numpy = self.title_embedding_transformer.encode(title_texts, convert_to_tensor=False)
            title_embeddings = torch.tensor(title_embeddings_numpy, dtype=torch.float).to(next(self.parameters()).device)
        return title_embeddings
    
    def forward(self, title_texts, target_tokens=None, max_len=100):
        """
        Forward pass through the model
        
        Args:
            title_texts: List of job title strings
            target_tokens: Tensor of target description tokens for teacher forcing [batch_size, seq_len]
            max_len: Maximum generation length for inference
            
        Returns:
            outputs: Either token predictions (for inference) or scores (for training)
        """
        batch_size = len(title_texts)
        device = next(self.parameters()).device
        
        # 1. Get title embeddings
        title_embeddings = self._get_title_embeddings(title_texts)  # [batch_size, embedding_dim]
        
        # 2. Project to initialize LSTM states (hidden and cell)
        h_0 = self.title_to_hidden(title_embeddings)  # [batch_size, hidden_dim]
        c_0 = self.title_to_cell(title_embeddings)    # [batch_size, hidden_dim]
        
        # Reshape for LSTM layers [num_layers, batch_size, hidden_dim]
        h_0 = h_0.unsqueeze(0).repeat(self.num_layers, 1, 1)
        c_0 = c_0.unsqueeze(0).repeat(self.num_layers, 1, 1)
        
        # 3. Run decoder (either with teacher forcing for training or step-by-step for inference)
        if target_tokens is not None:
            # Training mode with teacher forcing
            # Embed the target tokens
            embedded_targets = self.decoder_embedding(target_tokens)  # [batch_size, seq_len, decoder_embed_dim]
            embedded_targets = self.dropout(embedded_targets)
            
            # Pass through LSTM
            lstm_out, _ = self.lstm(embedded_targets, (h_0, c_0))  # [batch_size, seq_len, hidden_dim]
            
            # Project to vocabulary space
            output_logits = self.output_projection(lstm_out)  # [batch_size, seq_len, decoder_vocab_size]
            return output_logits
            
        else:
            # Inference mode - generate token by token
            outputs = []
            
            # Start with START token (assuming id=1, adjust as needed)
            current_token = torch.full((batch_size, 1), 1, dtype=torch.long, device=device)
            
            # Hidden states from title embedding
            hidden = (h_0, c_0)
            
            # Generate tokens step by step
            for _ in range(max_len):
                # Embed current token
                token_embedding = self.decoder_embedding(current_token)  # [batch_size, 1, decoder_embed_dim]
                
                # Get LSTM output for this step
                lstm_out, hidden = self.lstm(token_embedding, hidden)  # lstm_out: [batch_size, 1, hidden_dim]
                
                # Project to vocabulary
                logits = self.output_projection(lstm_out.squeeze(1))  # [batch_size, decoder_vocab_size]
                
                # Get the most likely next token
                _, predicted = torch.max(logits, dim=1)  # [batch_size]
                outputs.append(predicted.unsqueeze(1))
                
                # Use predicted token as next input
                current_token = predicted.unsqueeze(1)
                
                # Optional: stop if all sequences generate END token (id=2, adjust as needed)
                if (predicted == 2).all():
                    break
                    
            # Concatenate all predictions
            outputs = torch.cat(outputs, dim=1)  # [batch_size, seq_len]
            return outputs
    
    def train_model(self, dataloader, optimizer, criterion, num_epochs=5, clip_grad=1.0):
        """
        Train the model using the provided dataloader
        
        Args:
            dataloader: DataLoader yielding (title_embeddings, decoder_inputs, decoder_targets)
            optimizer: PyTorch optimizer
            criterion: Loss function (typically nn.CrossEntropyLoss)
            num_epochs: Number of training epochs
            clip_grad: Gradient clipping value
            
        Returns:
            losses: List of average losses per epoch
        """
        # This is the proper training loop, replacing the incomplete train method
        self.train()  # Set model to training mode
        
        device = next(self.parameters()).device
        losses = []
        
        for epoch in range(num_epochs):
            epoch_loss = 0
            epoch_items = 0
            
            for batch_idx, (title_texts, decoder_inputs, decoder_targets) in enumerate(dataloader):
                batch_size = decoder_inputs.size(0)
                
                # Forward pass
                outputs = self(title_texts, decoder_inputs)  # [batch_size, seq_len, vocab_size]
                
                # Reshape outputs and targets for the loss function
                outputs_flat = outputs.reshape(-1, self.decoder_vocab_size)  # [batch_size*seq_len, vocab_size]
                targets_flat = decoder_targets.reshape(-1)  # [batch_size*seq_len]
                
                # Calculate loss
                loss = criterion(outputs_flat, targets_flat)
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(self.parameters(), clip_grad)
                
                # Update weights
                optimizer.step()
                
                # Track statistics
                items = batch_size * decoder_targets.size(1)
                epoch_loss += loss.item() * items
                epoch_items += items
                
                if batch_idx % 50 == 0:
                    print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}")
            
            # Calculate average loss for the epoch
            avg_loss = epoch_loss / epoch_items
            losses.append(avg_loss)
            print(f"Epoch {epoch+1}/{num_epochs} complete, Avg Loss: {avg_loss:.4f}")
            
        return losses

In [ ]:

def get_pruned_vocab(df, column, top_k=50000):
    """
    Builds a vocabulary of the top_k most frequent tokens.
    """
    token_counter = Counter()
    for text in df[column]:
        tokens = text.split(" ")
        token_counter.update(tokens)

    most_common = token_counter.most_common(top_k)
    vocab = {token: idx for idx, (token, _) in enumerate(most_common)}
    vocab["__START__"] = len(vocab)
    vocab["__END__"] = len(vocab)
    vocab['<UNK>'] = len(vocab)
    vocab['<PAD>'] = len(vocab)
    return vocab, len(vocab)
    

In [ ]:
# 1. Define your special tokens and build vocabulary
special_tokens = ['<PAD>', '__START__', '__END__', '<UNK>']
# Build vocabulary from your dataset (count word frequencies, etc.)
list_of_common_words = get_pruned_vocab(df_train, "description")
word_to_idx = {token: idx for idx, token in enumerate(special_tokens + list_of_common_words)}
vocab_size = len(word_to_idx)

# 2. Create the dataloader (using your OptionBDataLoader class or similar)
cache_path = "unique_title_embeddings_lstm.pkl"
data_loader = OptionBDataLoader(
    dataframe=df,
    sentence_transformer_model=embedding_model,
    unique_titles_embedding_cache_path=cache_path,
    description_tokenizer_func=lambda x: x.lower().split(),  # Simple tokenizer
    description_word_to_idx=word_to_idx,
    batch_size=32,
    device=device
)

# 3. Create model instance
model = LSTMLanguageModelT(
    embedding_model=embedding_model,
    decoder_vocab_size=vocab_size,
    embedding_dim=384,  # For multi-qa-MiniLM-L6-cos-v1
    decoder_embed_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout_p=0.5
).to(device)

# 4. Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=word_to_idx['<PAD>'])  # Ignore padding tokens
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# 5. Train the model
losses = model.train_model(
    dataloader=data_loader.generate_batches(),
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=5
)

# 6. Generate a description from a title
def generate_description(model, title, max_len=100):
    model.eval()  # Set to evaluation mode
    with torch.no_grad():
        outputs = model([title], target_tokens=None, max_len=max_len)
        # Convert token IDs back to words
        idx_to_word = {idx: word for word, idx in word_to_idx.items()}
        words = [idx_to_word.get(idx.item(), '<UNK>') for idx in outputs[0]]
        # Remove any tokens after __END__
        if '__END__' in words:
            words = words[:words.index('__END__')]
        return ' '.join(words)

# Example: generate a description for a new title
sample_title = "Data Scientist"
generated_description = generate_description(model, sample_title)
print(f"Title: {sample_title}")
print(f"Generated Description: {generated_description}")